In [1]:
from scrollguard.utils import get_config
from scrollguard.etl import extract_csv, extract_xml, load_to_mongo
import requests
import ssl
import urllib3
import xmltodict
import pandas as pd

In [4]:
config = get_config()["SOURCES"]
data = extract_xml(config["CA_SANCTIONS"])

In [6]:
load_to_mongo("CA_SANCTIONS", data)